# Wasserstein GAN

ref.:
ARJOVSKY, Martin; CHINTALA, Soumith; BOTTOU, Léon.  
Wasserstein GAN.  
arXiv preprint arXiv:1701.07875, 2017.

![Comparing equations](equations.jpeg)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
import keras.backend as K

Using TensorFlow backend.


## Generator

In [3]:
def build_generator(latent_dim, channels):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=4, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

## Critic

In [4]:
def build_critic(img_shape):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

## Helper functions

In [5]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [6]:
def sample_images(G, latent_dim, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [7]:
def train(G, critic, combined, n_critic, latent_dim, clip_value, epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = -np.ones((batch_size, 1))
    fake = np.ones((batch_size, 1))

    for epoch in range(epochs):

        for _ in range(n_critic):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, latent_dim))

            # Generate a batch of new images
            gen_imgs = G.predict(noise)

            # Train the critic
            d_loss_real = critic.train_on_batch(imgs, valid)
            d_loss_fake = critic.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

            # Clip critic weights
            for l in critic.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                l.set_weights(weights)


        # ---------------------
        #  Train Generator
        # ---------------------

        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, latent_dim, epoch)

## main()

In [8]:
if not os.path.exists('images'):
    os.makedirs('images')

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [10]:
# Following parameter and optimizer set as recommended in paper
n_critic = 5
clip_value = 0.01

In [11]:
# create optimizer
optimizer = RMSprop(lr=0.00005)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [12]:
# Build and compile the critic
critic = build_critic(img_shape)
critic.compile(loss=wasserstein_loss,
               optimizer=optimizer,
               metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)         

In [13]:
# Build the generator
G = build_generator(latent_dim, channels)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)      

In [14]:
# The generator takes noise as input and generated imgs
z = Input(shape=(latent_dim,))
img = G(z)

In [15]:
# For the combined model we will only train the generator
critic.trainable = False

In [16]:
# The critic takes generated images as input and determines validity
valid = critic(img)

In [17]:
# The combined model  (stacked generator and critic)
combined = Model(z, valid)
combined.compile(loss=wasserstein_loss,
                 optimizer=optimizer,
                 metrics=['accuracy'])

In [ ]:
# epochs=4000
epochs=1000
train(G, critic, combined, 
      n_critic, latent_dim, clip_value,
      epochs=epochs, batch_size=32, sample_interval=50)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.999930] [G loss: 1.000140]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.999940] [G loss: 1.000139]
2 [D loss: 0.999936] [G loss: 1.000137]
3 [D loss: 0.999940] [G loss: 1.000130]
4 [D loss: 0.999935] [G loss: 1.000134]
5 [D loss: 0.999939] [G loss: 1.000140]
6 [D loss: 0.999938] [G loss: 1.000126]
7 [D loss: 0.999937] [G loss: 1.000127]
8 [D loss: 0.999940] [G loss: 1.000131]
9 [D loss: 0.999938] [G loss: 1.000130]
10 [D loss: 0.999937] [G loss: 1.000122]
11 [D loss: 0.999937] [G loss: 1.000124]
12 [D loss: 0.999943] [G loss: 1.000124]
13 [D loss: 0.999942] [G loss: 1.000119]
14 [D loss: 0.999940] [G loss: 1.000117]
15 [D loss: 0.999946] [G loss: 1.000115]
16 [D loss: 0.999948] [G loss: 1.000118]
17 [D loss: 0.999946] [G loss: 1.000111]
18 [D loss: 0.999950] [G loss: 1.000109]
19 [D loss: 0.999948] [G loss: 1.000109]
20 [D loss: 0.999948] [G loss: 1.000102]
21 [D loss: 0.999957] [G loss: 1.000100]
22 [D loss: 0.999955] [G loss: 1.000095]
23 [D loss: 0.999953] [G loss: 1.000095]
24 [D loss: 0.999959] [G loss: 1.000092]
25 [D loss: 0.999956] [G 

199 [D loss: 0.999968] [G loss: 1.000077]
200 [D loss: 0.999974] [G loss: 1.000064]
201 [D loss: 0.999961] [G loss: 1.000057]
202 [D loss: 0.999962] [G loss: 1.000066]
203 [D loss: 0.999967] [G loss: 1.000068]
204 [D loss: 0.999968] [G loss: 1.000070]
205 [D loss: 0.999969] [G loss: 1.000070]
206 [D loss: 0.999976] [G loss: 1.000064]
207 [D loss: 0.999969] [G loss: 1.000070]
208 [D loss: 0.999962] [G loss: 1.000071]
209 [D loss: 0.999971] [G loss: 1.000052]
210 [D loss: 0.999974] [G loss: 1.000057]
211 [D loss: 0.999968] [G loss: 1.000056]
212 [D loss: 0.999968] [G loss: 1.000077]
213 [D loss: 0.999960] [G loss: 1.000064]
214 [D loss: 0.999967] [G loss: 1.000068]
215 [D loss: 0.999969] [G loss: 1.000076]
216 [D loss: 0.999971] [G loss: 1.000065]
217 [D loss: 0.999970] [G loss: 1.000060]
218 [D loss: 0.999971] [G loss: 1.000071]
219 [D loss: 0.999978] [G loss: 1.000061]
220 [D loss: 0.999958] [G loss: 1.000079]
221 [D loss: 0.999974] [G loss: 1.000091]
222 [D loss: 0.999979] [G loss: 1.

395 [D loss: 0.999969] [G loss: 1.000058]
396 [D loss: 0.999976] [G loss: 1.000054]
397 [D loss: 0.999963] [G loss: 1.000064]
398 [D loss: 0.999979] [G loss: 1.000071]
399 [D loss: 0.999955] [G loss: 1.000065]
400 [D loss: 0.999973] [G loss: 1.000071]
401 [D loss: 0.999966] [G loss: 1.000064]
402 [D loss: 0.999968] [G loss: 1.000068]
403 [D loss: 0.999968] [G loss: 1.000063]
404 [D loss: 0.999976] [G loss: 1.000062]
405 [D loss: 0.999962] [G loss: 1.000075]
406 [D loss: 0.999975] [G loss: 1.000065]
407 [D loss: 0.999969] [G loss: 1.000061]
408 [D loss: 0.999973] [G loss: 1.000062]
409 [D loss: 0.999967] [G loss: 1.000068]
410 [D loss: 0.999974] [G loss: 1.000065]
411 [D loss: 0.999965] [G loss: 1.000054]
412 [D loss: 0.999978] [G loss: 1.000064]
413 [D loss: 0.999969] [G loss: 1.000068]
414 [D loss: 0.999965] [G loss: 1.000062]
415 [D loss: 0.999957] [G loss: 1.000066]
416 [D loss: 0.999974] [G loss: 1.000054]
417 [D loss: 0.999974] [G loss: 1.000070]
418 [D loss: 0.999971] [G loss: 1.

591 [D loss: 0.999970] [G loss: 1.000065]
592 [D loss: 0.999968] [G loss: 1.000066]
593 [D loss: 0.999975] [G loss: 1.000069]
594 [D loss: 0.999968] [G loss: 1.000062]
595 [D loss: 0.999970] [G loss: 1.000075]
596 [D loss: 0.999967] [G loss: 1.000071]
597 [D loss: 0.999978] [G loss: 1.000055]
598 [D loss: 0.999970] [G loss: 1.000066]
599 [D loss: 0.999974] [G loss: 1.000065]
600 [D loss: 0.999965] [G loss: 1.000076]
601 [D loss: 0.999963] [G loss: 1.000062]
602 [D loss: 0.999968] [G loss: 1.000066]
603 [D loss: 0.999972] [G loss: 1.000066]
604 [D loss: 0.999966] [G loss: 1.000062]
605 [D loss: 0.999979] [G loss: 1.000061]
606 [D loss: 0.999976] [G loss: 1.000071]
607 [D loss: 0.999963] [G loss: 1.000048]
608 [D loss: 0.999967] [G loss: 1.000058]
609 [D loss: 0.999973] [G loss: 1.000058]
610 [D loss: 0.999959] [G loss: 1.000061]
611 [D loss: 0.999969] [G loss: 1.000052]
612 [D loss: 0.999976] [G loss: 1.000068]
613 [D loss: 0.999967] [G loss: 1.000063]
614 [D loss: 0.999980] [G loss: 1.